In [9]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

np.random.seed(42)

N = 100
p = 8 # predictores

# Generar X aleatorio
X = np.random.normal(0, 1, (N, p))

# Verdaderos coeficientes: solo los primeros 3 importan
beta_true = np.array([2.0, -1.5, 3.0, 0, 0, 0, 0, 0])
intercept_true = 1.0

y = intercept_true + X @ beta_true + np.random.normal(0, 2.0, N)

# Agregar columna de intercepto
X_full = np.column_stack((np.ones(N), X)) # shape (100, 9)


In [10]:
def best_subset(X, y, max_k=None):
    """
    Best subset selection exhaustiva.
    Tegnologia inversa: probamos todas las combinaciones de k variables,
    ajustamos OLS y guardamos RSS, Cp, AIC, BIC.
    """
    n, p = X.shape
    if max_k is None:
        max_k = p - 1 # hasta p-1 (sin intercepto)
    results = []

    for k in range(1, max_k + 1):
        best_rss = np.inf
        best_subset_idx = None

        # Todas las convinaciones de k variables (sin intercepto)
        for subset in combinations(range(1, p), k):  # range(1,p) excluye intercepto
            cols = [0] + list(subset)  # siempre incluimos intercepto
            X_sub = X[:, cols]
            betas = np.linalg.lstsq(X_sub, y, rcond=None)[0]
            y_pred = X_sub @ betas
            rss = np.sum((y - y_pred)**2)

            if rss < best_rss:
                best_rss = rss
                best_subset_idx = subset
        # Calculos de criterio (k total = k + 1 por intercepto)
        k_total = k + 1
        aic = n * np.log(best_rss / n) + 2 * k_total
        bic = n * np.log(best_rss / n) + np.log(n) * k_total
        cp = best_rss / np.var(y) + 2 * k_total - p

        results.append({
            'k': k,
            'rss': best_rss,
            'aic': aic,
            'bic': bic,
            'cp': cp,
            'subset': best_subset_idx
        })
    return results        

In [11]:
results = best_subset(X_full, y, max_k=5)  # hasta k=5 para no tardar mucho

print("k | RSS | Cp | AIC | BIC | Mejores predictores")
print("-" * 60)
for r in results:
    print(f"{r['k']} | {r['rss']:.2f} | {r['cp']:.2f} | {r['aic']:.2f} | {r['bic']:.2f} | {r['subset']}")

k | RSS | Cp | AIC | BIC | Mejores predictores
------------------------------------------------------------
1 | 760.18 | 42.14 | 206.84 | 212.05 | (3,)
2 | 470.63 | 26.19 | 160.89 | 168.71 | (1, 3)
3 | 339.94 | 20.08 | 130.36 | 140.78 | (1, 2, 3)
4 | 333.65 | 21.69 | 130.49 | 143.52 | (1, 2, 3, 7)
5 | 331.77 | 23.57 | 131.93 | 147.56 | (1, 2, 3, 7, 8)
